In [46]:
import sys
import numpy as np
import pandas as pd
from statsmodels.tsa.arima_model import ARIMA

sys.path.append('./sources')
from models import predict_knn, predict_LDMM, predict_SAME
%load_ext autoreload
%autoreload 2
# from ldmm import LDMM
# from same import SAME

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Loading data

In [47]:
data = pd.read_csv('data/data.csv')

In [48]:
data = data.to_numpy()[:,1:].astype(np.float)

In [49]:
#Excluding seasonability 
timestamps = np.arange(data.shape[0])
mask = np.tile(np.append(np.ones(1), np.append(np.zeros(10), np.ones(1))), 20)[:-2]
timestamps = timestamps[mask==0]
data = data[mask==0]

### Setting parameters

In [50]:
num_series = data.shape[1]
# Number of test months
n_test = 40
# LDMM parameters
bandwidth_ldmm, lambd, mu, h, n_iter_ldmm = 11, 7.0, 1500.0, 0.001, 7
nn_ldmm = [60, 10, 7, 7]
# SAME parameters
bandwidth_same, tau, n_iter_same = 3, 1.0, 21
nn_same = [9, 21, 21, 15]
# kNN parameters
bandwidth_knn = 12
nn_knn = [5, 30, 30, 30]

In [55]:
losses = []
for lookfront in [1,2,3,4]:
    predictions_knn   = np.empty((n_test, lookfront, num_series))
    predictions_ldmm  = np.empty((n_test, lookfront, num_series))
    predictions_same  = np.empty((n_test, lookfront, num_series))
    predictions_arima = np.empty((n_test, lookfront, num_series))

    outcomes = np.empty((n_test, lookfront, num_series))
    for i in range(n_test):

        Y_train_knn   = data[:-n_test+i+1-lookfront, :]
        Y_train_ldmm  = data[:-n_test+i+1-lookfront, :]
        Y_train_same  = data[:-n_test+i+1-lookfront, :]
        Y_train_arima = data[:-n_test+i+1-lookfront, :]

        for k in range(lookfront):
            timestamps_train = timestamps[:-n_test+i+1-lookfront+k]
            timestamp_test = timestamps[-n_test+i+1-lookfront+k]
            Y_test = data[-n_test+i+1-lookfront+k, :]

            predictions_knn[i, k, :] = predict_knn(timestamps_train, Y_train_ldmm, timestamp_test,\
                                                   bandwidth_knn, nn_knn)[:]
            predictions_ldmm[i, k, :] = predict_LDMM(timestamps_train, Y_train_ldmm, timestamp_test,\
                                                    bandwidth_ldmm, lambd, mu, h, n_iter_ldmm, nn_ldmm)[:]
            predictions_same[i, k, :] = predict_SAME(timestamps_train, Y_train_same, timestamp_test,\
                                                    bandwidth_same, tau, n_iter_same, nn_same)[:]
            for t in range(num_series):
                arima_model = ARIMA(Y_train_arima[:, t], order=(6,1,0))
                predictions_arima[i, k, t] = arima_model.fit(disp=0, trend='nc').forecast(steps=1)[0] 


            outcomes[i, k, :] = Y_test[:]
            Y_train_knn = np.append(Y_train_knn, predictions_knn[i, k, :].reshape(1,-1), axis=0)
            Y_train_ldmm = np.append(Y_train_ldmm, predictions_ldmm[i, k, :].reshape(1,-1), axis=0)
            Y_train_same = np.append(Y_train_same, predictions_same[i, k, :].reshape(1,-1), axis=0)
            Y_train_arima = np.append(Y_train_arima, predictions_arima[i, k, :].reshape(1,-1), axis=0)


    new_loss_knn = np.mean((predictions_knn[:,-1]-outcomes[:,-1])**2/ outcomes[:,-1]**2, axis=0)
    new_loss_ldmm = np.mean((predictions_ldmm[:,-1]-outcomes[:,-1])**2 / outcomes[:,-1]**2, axis=0)
    new_loss_same = np.mean((predictions_same[:,-1]-outcomes[:,-1])**2 / outcomes[:,-1]**2, axis=0)
    new_loss_arima = np.mean((predictions_arima[:,-1]-outcomes[:,-1])**2 / outcomes[:,-1]**2, axis=0)

    print(lookfront)
    print('knn', new_loss_knn)
    print('ldmm', new_loss_ldmm)
    print('same', new_loss_same)
    print('arima', new_loss_arima)

    print()


1
knn [0.00165557 0.00396715 0.00239861 0.00177703]
ldmm [0.00155454 0.00241133 0.00152581 0.00142244]
same [0.00116738 0.0018553  0.00151837 0.0016329 ]
arima [0.00837624 0.00213298 0.00145582 0.00158337]

2
knn [0.00176472 0.00434126 0.00303796 0.00427745]
ldmm [0.0014592 0.0028675 0.00198019 0.0042891]
same [0.001458 0.00245318 0.00209719 0.00394271]
arima [0.00176388 0.00249325 0.00194276 0.00375188]

3
knn [0.0030487  0.00709648 0.0042467  0.00584049]
ldmm [0.00281025 0.00402681 0.00293258 0.00486037]
same [0.00189623 0.00334985 0.00278699 0.00510701]
arima [0.00183599 0.0031047  0.00202661 0.00488685]

4
knn [0.00314675 0.00723589 0.00450714 0.0060282 ]
ldmm [0.00304857 0.00387206 0.00345893 0.00501547]
same [0.0020379  0.00360712 0.00364615 0.00557899]
arima [0.00199373 0.00331188 0.00212744 0.00518051]

